In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from transformers import pipeline, GPT2Tokenizer, GPT2LMHeadModel
import json

In [ ]:
with open('/content/drive/MyDrive/Internship/Actionfi/intents copy new.json', 'r') as json_file:
    intents = json.load(json_file)

data = pd.DataFrame(intents['intents'])
data

,tag,patterns,responses,response
0,greeting,"[Hi, Hey, Is anyone there?, Hi there, Hello, H...",[Hello there. Tell me how are you feeling toda...,NaN
1,morning,[Good morning],[Good morning. I hope you had a good night's s...,NaN
2,afternoon,[Good afternoon],[Good afternoon. How is your day going?],NaN
3,evening,[Good evening],[Good evening. How has your day been?],NaN
4,night,[Good night],"[Good night. Get some proper sleep, Good night...",NaN
...,...,...,...,...
80,fact-28,[What do I do if I'm worried about my mental h...,[The most important thing is to talk to someon...,NaN
81,fact-29,[How do I know if I'm unwell?],"[If your beliefs , thoughts , feelings or beha...",NaN
82,fact-30,[How can I maintain social connections? What i...,"[A lot of people are alone right now, but we d...",NaN
83,fact-31,[What's the difference between anxiety and str...,[Stress and anxiety are often used interchange...,NaN


In [ ]:
dic = {"tag":[], "patterns":[], "responses":[]}
for i in range(len(data)):
    ptrns = data[data.index == i]['patterns'].values[0]
    rspns = data[data.index == i]['responses'].values[0]
    tag = data[data.index == i]['tag'].values[0]
    for j in range(len(ptrns)):
        dic['tag'].append(tag)
        dic['patterns'].append(ptrns[j])
        dic['responses'].append(rspns)

data = pd.DataFrame.from_dict(dic)
data

,tag,patterns,responses
0,greeting,Hi,[Hello there. Tell me how are you feeling toda...
1,greeting,Hey,[Hello there. Tell me how are you feeling toda...
2,greeting,Is anyone there?,[Hello there. Tell me how are you feeling toda...
3,greeting,Hi there,[Hello there. Tell me how are you feeling toda...
4,greeting,Hello,[Hello there. Tell me how are you feeling toda...
...,...,...,...
241,fact-29,How do I know if I'm unwell?,"[If your beliefs , thoughts , feelings or beha..."
242,fact-30,How can I maintain social connections? What if...,"[A lot of people are alone right now, but we d..."
243,fact-31,What's the difference between anxiety and stress?,[Stress and anxiety are often used interchange...
244,fact-32,What's the difference between sadness and depr...,"[Sadness is a normal reaction to a loss, disap..."


In [ ]:
label_encoder = LabelEncoder()
data['tag_encoded'] = label_encoder.fit_transform(data['tag'])

In [ ]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(data['patterns'])
X = tokenizer.texts_to_sequences(data['patterns'])
X = pad_sequences(X, padding='post')

In [ ]:
print(X)

[[ 96   0   0 ...   0   0   0]
 [ 97   0   0 ...   0   0   0]
 [ 15  98  41 ...   0   0   0]
 ...
 [ 78  16  69 ...   0   0   0]
 [ 78  16  69 ...   0   0   0]
 [ 69  70 157 ...   0   0   0]]


In [ ]:
y = data['tag_encoded']

In [ ]:
print(y)

0      46
1      46
2      46
3      46
4      46
       ..
241    34
242    36
243    37
244    38
245    38
Name: tag_encoded, Length: 246, dtype: int64


In [ ]:
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=64, input_length=X.shape[1]))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(X, y, epochs=50, batch_size=32)


Epoch 1/50
8/8 [==============================] - 6s 55ms/step - loss: 4.4373 - accuracy: 0.0325
Epoch 2/50
8/8 [==============================] - 0s 47ms/step - loss: 4.4042 - accuracy: 0.0488
Epoch 3/50
8/8 [==============================] - 0s 29ms/step - loss: 4.3258 - accuracy: 0.0488
Epoch 4/50
8/8 [==============================] - 0s 26ms/step - loss: 4.1600 - accuracy: 0.0488
Epoch 5/50
8/8 [==============================] - 0s 27ms/step - loss: 4.1115 - accuracy: 0.0488
Epoch 6/50
8/8 [==============================] - 0s 26ms/step - loss: 4.0556 - accuracy: 0.0610
Epoch 7/50
8/8 [==============================] - 0s 27ms/step - loss: 3.9905 - accuracy: 0.0650
Epoch 8/50
8/8 [==============================] - 0s 32ms/step - loss: 3.9046 - accuracy: 0.0610
Epoch 9/50
8/8 [==============================] - 0s 25ms/step - loss: 3.8043 - accuracy: 0.0813
Epoch 10/50
8/8 [==============================] - 0s 26ms/step - loss: 3.7250 - accuracy: 0.0732
Epoch 11/50
8/8 [============

In [43]:
generator = pipeline('text-generation', model='gpt2')


In [ ]:
class ChatBot:
    def __init__(self, model, tokenizer, label_encoder, data_frame, generator):
        self.model = model
        self.tokenizer = tokenizer
        self.label_encoder = label_encoder
        self.data_frame = data_frame
        self.generator = generator

    def preprocess(self, pattern):
        pattern_seq = self.tokenizer.texts_to_sequences([pattern])
        pattern_pad = pad_sequences(pattern_seq, maxlen=X.shape[1], padding='post')
        return pattern_pad

    def predict_tag(self, pattern):
        pattern_pad = self.preprocess(pattern)
        predicted_prob = self.model.predict(pattern_pad)[0]
        predicted_tag_index = np.argmax(predicted_prob)
        predicted_tag = self.label_encoder.inverse_transform([predicted_tag_index])[0]
        return predicted_tag

    def get_response(self, predicted_tag):
        responses = self.data_frame[self.data_frame['tag'] == predicted_tag]['responses'].values
        if len(responses) > 0:
            return responses[0]
        else:
            return None

    def generate_response(self, pattern):
        generated_response = self.generator(pattern, max_length=50, num_return_sequences=1)[0]['generated_text']
        return generated_response

    def respond(self, pattern):
        predicted_tag = self.predict_tag(pattern)
        response = self.get_response(predicted_tag)
        if response:
          print(f"The predicted tag is of response {predicted_tag}")
          return response[0] if isinstance(response, list) else response
        else:
          print(f"The predicted tag is of pattern {predicted_tag}")
          return self.generate_response(pattern)

In [ ]:
chatbot = ChatBot(model, tokenizer, label_encoder, data, generator)


In [ ]:
test_pattern = "Week was tough. But I made through"
response = chatbot.respond(test_pattern)
print("ChatBot:", response)

1/1 [==============================] - 0s 21ms/step
The predicted tag is of response worthless
ChatBot: It's only natural to feel this way. Tell me more. What else is on your mind?


In [ ]:
print("Amvi: Hi! How can I assist you today?")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Amvi: Goodbye! Take care.")
        break
    response = chatbot.respond(user_input)
    print(f"Amvi: {response}")

Amvi: Hi! How can I assist you today?
You: hi
1/1 [==============================] - 0s 23ms/step
The predicted tag is of response greeting
Amvi: Hello there. Tell me how are you feeling today?
You: I am not feeling good, Very tired
1/1 [==============================] - 0s 22ms/step
The predicted tag is of response alone
Amvi: Just remember you are not alone. I am am here for you
You: bye
Amvi: Goodbye! Take care.
